In [2]:
import os
import openai
from dotenv import load_dotenv

# API 키를 환경 변수에서 로드 (보안상 더 좋은 방법)
# .env 파일이 없을 경우 직접 설정
try:
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    
finally:
    print(f"OpenAI 라이브러리 버전: {openai.__version__}")

OpenAI 라이브러리 버전: 1.79.0


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=1.0,
    max_tokens=2048,

    model_name="gpt-4o-mini"
)

In [4]:
from langchain.prompts import ChatPromptTemplate

# 시스템 프롬프트 : LLM에게 특정 역할이나 행동 방식을 지정하는 역할
sys_template = "당신은 게임 JSON 파일을 제작하는 전문가입니다. 상세하고 일관성 있는 게임 데이터를 생성합니다."
user_template = "{question}"

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", sys_template),
    ("user", user_template)
])

# LLM 모델 설정 - 일관된 결과를 위해 temperature=0 사용
llm = ChatOpenAI(
     model='gpt-4o-mini',
     temperature=0
)

In [5]:
# 상세 게임 시나리오 및 JSON 생성 요청 정의
game_scenario_prompt = """
다음은 '아기돼지 삼형제 주식회사' 투자 시뮬레이션 게임의 전체 시나리오입니다. 
이 설명을 바탕으로, 게임의 각 턴별 상황을 상세한 JSON 형식으로 생성해주세요. 
JSON은 총 10개 턴의 정보를 담고 있는 리스트여야 합니다. 
각 턴 객체는 다음 키를 가져야 합니다:
- `turn_number`: (정수) 현재 턴 번호 (1부터 10까지).
- `news`: (문자열) 해당 턴에 플레이어에게 제공되는 뉴스 또는 힌트.
- `event_description`: (문자열) 해당 턴에 발생하는 주요 이벤트 설명. 이벤트가 없다면 "없음".
- `stocks`: (리스트) 각 투자 종목의 상세 정보를 담은 객체들의 리스트. 각 종목 객체는 다음 키를 가져야 합니다:
    - `name`: (문자열) 종목 이름 (예: "첫째집", "둘째집", "셋째집").
    - `description`: (문자열) 종목에 대한 간략한 설명 및 특징.
    - `initial_value`: (정수) 게임 시작 시점의 초기 가치 (모든 턴에서 동일하게 100으로 표시).
    - `current_value`: (정수) 해당 턴 종료 시점의 현재 가치. 이 값은 뉴스와 이벤트를 반영하여 논리적으로 변동되어야 합니다.
    - `risk_level`: (문자열) 해당 종목의 위험도 (예: "고위험 고수익", "균형형", "장기 투자 적합").

**게임 배경 및 주요 정보:**
- **주제:** 어린이 눈높이의 투자 시뮬레이션 - '아기돼지 삼형제 주식회사'
- **투자 종목:**
    1.  **첫째집 (지푸라기 집):** 초기 가치 100, 빠른 완공, 인기도 높으나 리스크 큼 (고위험 고수익).
    2.  **둘째집 (나무 집):** 초기 가치 100, 중간 정도의 안정성과 속도 (균형형).
    3.  **셋째집 (벽돌 집):** 초기 가치 100, 느리지만 안정성 최고 (장기 투자 적합).

**턴별 시나리오 가이드라인 (총 10턴):**

*   **1턴:**
    *   뉴스: "이번 주 날씨는 맑음! 첫째 돼지의 지푸라기 집이 빠르게 완성되어 인기가 치솟고 있어요!"
    *   이벤트: 없음.
    *   가치 변동 예상: 첫째집 가치 소폭 상승 (예: +5~10%). 다른 집 변동 없음.

*   **2턴:**
    *   뉴스: "일기예보: 2턴 뒤에 강력한 태풍이 몰려올 수 있다는 소식입니다! 투자에 유의하세요."
    *   이벤트: 없음.
    *   가치 변동 예상: 전체적으로 관망세. 큰 변동 없을 수 있으나, 투자자들은 태풍 소식에 주목.

*   **3턴:**
    *   뉴스: "긴급 속보: 태풍이 예상보다 빠르게 북상 중! 특히 둘째 돼지의 나무 집이 강풍에 취약할 수 있다는 분석이 나왔습니다."
    *   이벤트: 없음.
    *   가치 변동 예상: 둘째집 가치 소폭 하락 우려 (예: -5~10%). 셋째집에 대한 관심 증가로 소폭 상승 가능성 (예: +5%). 첫째집은 여전히 리스크와 인기도 공존.

*   **4턴:**
    *   뉴스: "태풍이 드디어 상륙했습니다! 거센 비바람이 몰아치고 있습니다."
    *   이벤트: "초대형 태풍 발생! 둘째 돼지의 나무 집이 심각한 피해를 입어 가치가 폭락했습니다! 첫째 돼지의 지푸라기 집도 일부 손상되었습니다."
    *   가치 변동 예상:
        *   둘째집: 가치 대폭 하락 (예: -80% ~ -90%), 거의 0에 가깝거나 거래 불가 상태 가능.
        *   첫째집: 태풍 피해로 가치 하락 (예: -20% ~ -40%).
        *   셋째집: 태풍을 잘 견뎌내어 가치 유지 또는 안전자산으로 부각되며 소폭 상승 (예: +5~10%).

*   **5턴:**
    *   뉴스: "태풍이 지나갔습니다. 셋째 돼지의 벽돌 집은 튼튼함을 증명하며 가치가 더욱 상승했습니다! 반면, 다른 집들은 피해 복구에 시간이 걸릴 것 같습니다."
    *   이벤트: "셋째집 안전성 재평가."
    *   가치 변동 예상:
        *   셋째집: 가치 크게 상승 (예: +20~30%).
        *   첫째집: 피해 상황에 따라 소폭 회복 시도 또는 낮은 가치 유지.
        *   둘째집: 여전히 매우 낮은 가치 또는 거래 불가.

위 가이드라인을 따라서, 각 턴의 `current_value`가 이전 턴의 상황, 뉴스, 이벤트를 종합적으로 고려하여 현실감 있게 변동되도록 JSON을 생성해주세요.
반드시 전체 응답이 하나의 JSON 리스트 형식이어야 합니다. 다른 설명 없이 JSON 데이터만 출력해주세요.
"""

# LangChain 체인 실행
print("게임 시나리오 데이터 생성 중...")
chain = prompt | llm
response = chain.invoke({"question": game_scenario_prompt})

print("\n생성된 게임 시나리오 JSON:")
print(response.content)

# JSON 문자열을 파싱하여 Python 객체로 변환하고 예쁘게 출력
import json
import pprint
try:
    game_data = json.loads(response.content)
    print("\n파싱된 데이터 (Pretty Print):")
    pprint.pprint(game_data, indent=2)
    
    # 데이터가 성공적으로 파싱되었으면 변수에 저장
    parsed_game_data = game_data
    print("\n데이터가 성공적으로 파싱되었습니다.")
    
except json.JSONDecodeError as e:
    print(f"\nJSON 파싱 오류: {e}")
    print("LLM의 원본 응답 내용을 확인하세요. 완전한 JSON 형식이 아닐 수 있습니다.")
    parsed_game_data = None
except Exception as e:
    print(f"\n데이터 처리 중 예기치 않은 오류 발생: {e}")
    parsed_game_data = None

게임 시나리오 데이터 생성 중...

생성된 게임 시나리오 JSON:
```json
[
    {
        "turn_number": 1,
        "news": "이번 주 날씨는 맑음! 첫째 돼지의 지푸라기 집이 빠르게 완성되어 인기가 치솟고 있어요!",
        "event_description": "없음",
        "stocks": [
            {
                "name": "첫째집",
                "description": "지푸라기 집, 빠른 완공과 높은 인기로 인한 인기 상승.",
                "initial_value": 100,
                "current_value": 108,
                "risk_level": "고위험 고수익"
            },
            {
                "name": "둘째집",
                "description": "나무 집, 중간 정도의 안정성과 속도를 가진 집.",
                "initial_value": 100,
                "current_value": 100,
                "risk_level": "균형형"
            },
            {
                "name": "셋째집",
                "description": "벽돌 집, 느리지만 안정성이 가장 높은 집.",
                "initial_value": 100,
                "current_value": 100,
                "risk_level": "장기 투자 적합"
            }
        ]
    },
    {
        "turn_number": 2,
        "news": "일기예보: 2턴 뒤에 강력한 태풍

In [6]:
# 생성된 JSON 데이터를 파일로 저장
import os

def save_game_data(data, filename="game_scenario.json"):
    """
    게임 데이터를 JSON 파일로 저장하는 함수
    
    Args:
        data: 저장할 게임 데이터
        filename: 저장할 파일 이름 (기본값: game_scenario.json)
    
    Returns:
        성공 여부 메시지
    """
    if data is None:
        return "저장할 데이터가 없습니다. JSON 파싱 오류를 확인하세요."
    
    try:
        save_path = os.path.join(os.path.dirname(os.path.abspath("__file__")), filename)
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        return f"데이터가 성공적으로 {save_path}에 저장되었습니다."
    except Exception as e:
        return f"데이터 저장 중 오류 발생: {e}"

# 파싱된 데이터가 있으면 파일로 저장
if 'parsed_game_data' in locals() and parsed_game_data is not None:
    result = save_game_data(parsed_game_data)
    print(result)

In [7]:
# 게임 데이터 시각화 - 턴별 주식 가치 변동 그래프
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def visualize_stock_values(game_data):
    """
    게임 데이터에서 턴별 주식 가치 변동을 시각화하는 함수
    
    Args:
        game_data: 시각화할 게임 데이터
    """
    if game_data is None:
        print("시각화할 데이터가 없습니다.")
        return
    
    try:
        # 데이터 준비
        turns = []
        first_house_values = []
        second_house_values = []
        third_house_values = []
        
        for turn in game_data:
            turns.append(turn['turn_number'])
            
            # 각 집의 값을 찾아 저장
            for stock in turn['stocks']:
                if stock['name'] == "첫째집":
                    first_house_values.append(stock['current_value'])
                elif stock['name'] == "둘째집":
                    second_house_values.append(stock['current_value'])
                elif stock['name'] == "셋째집":
                    third_house_values.append(stock['current_value'])
        
        # 데이터프레임으로 변환
        df = pd.DataFrame({
            'Turn': turns,
            '첫째집 (지푸라기 집)': first_house_values,
            '둘째집 (나무 집)': second_house_values,
            '셋째집 (벽돌 집)': third_house_values
        })
        
        # 시각화
        plt.figure(figsize=(12, 6))
        plt.plot(df['Turn'], df['첫째집 (지푸라기 집)'], 'o-', color='gold', label='첫째집 (지푸라기 집)')
        plt.plot(df['Turn'], df['둘째집 (나무 집)'], 'o-', color='brown', label='둘째집 (나무 집)')
        plt.plot(df['Turn'], df['셋째집 (벽돌 집)'], 'o-', color='firebrick', label='셋째집 (벽돌 집)')
        
        # 초기 가치 기준선 추가
        plt.axhline(y=100, color='gray', linestyle='--', alpha=0.7, label='초기 가치')
        
        # 그래프 꾸미기
        plt.title('아기돼지 삼형제 주식회사 - 턴별 주식 가치 변동', fontsize=16)
        plt.xlabel('턴 (Turn)', fontsize=12)
        plt.ylabel('주식 가치', fontsize=12)
        plt.xticks(turns)
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # 중요 이벤트 표시
        for turn in game_data:
            if turn['event_description'] != "없음":
                plt.annotate(f"이벤트: 턴 {turn['turn_number']}", 
                             xy=(turn['turn_number'], 50),
                             xytext=(turn['turn_number'], 20),
                             arrowprops=dict(facecolor='black', shrink=0.05, width=1.5),
                             fontsize=9,
                             horizontalalignment='center')
        
        plt.tight_layout()
        plt.show()
        
        # 턴별 뉴스와 이벤트 정보 출력
        print("\n턴별 뉴스 및 이벤트 정보:")
        for turn in game_data:
            print(f"\n[턴 {turn['turn_number']}]")
            print(f"뉴스: {turn['news']}")
            print(f"이벤트: {turn['event_description'] if turn['event_description'] != '없음' else '이벤트 없음'}")
            
    except Exception as e:
        print(f"시각화 중 오류 발생: {e}")

# 파싱된 데이터가 있으면 시각화 실행
if 'parsed_game_data' in locals() and parsed_game_data is not None:
    visualize_stock_values(parsed_game_data)

In [8]:
# 인터랙티브 게임 시뮬레이션 (옵션)
# 사용자가 턴별로 의사결정을 내릴 수 있는 간단한 시뮬레이션

def run_simulation(game_data):
    """
    게임 데이터를 기반으로 간단한 투자 시뮬레이션을 실행하는 함수
    
    Args:
        game_data: 시뮬레이션에 사용할 게임 데이터
    """
    if game_data is None:
        print("시뮬레이션을 실행할 데이터가 없습니다.")
        return
    
    try:
        print("\n===== 아기돼지 삼형제 주식회사 투자 시뮬레이션 =====\n")
        print("각 턴마다 세 종목('첫째집', '둘째집', '셋째집') 중 하나에 투자할 수 있습니다.")
        print("시작 자본금은 1000원이며, 턴이 끝날 때마다 투자한 종목의 수익률에 따라 자본금이 변동됩니다.\n")
        
        capital = 1000  # 초기 자본금
        
        for turn in game_data:
            print(f"\n===== 턴 {turn['turn_number']} =====")
            print(f"현재 자본금: {capital}원")
            print(f"\n뉴스: {turn['news']}")
            
            # 종목 정보 출력
            print("\n현재 종목 정보:")
            for stock in turn['stocks']:
                print(f"- {stock['name']}: {stock['current_value']} (위험도: {stock['risk_level']})")
            
            # 사용자 입력 받기
            valid_stocks = [stock['name'] for stock in turn['stocks']]
            while True:
                choice = input(f"\n어떤 종목에 투자하시겠습니까? ({', '.join(valid_stocks)}, 또는 '패스'): ")
                if choice in valid_stocks or choice == '패스':
                    break
                print("잘못된 입력입니다. 다시 시도해주세요.")
            
            # 투자 결과 계산
            if choice == '패스':
                print("이번 턴은 투자를 패스합니다.")
            else:
                # 다음 턴의 해당 종목 가치 찾기 (마지막 턴이면 현재 턴 사용)
                next_turn_index = turn['turn_number'] if turn['turn_number'] == len(game_data) else turn['turn_number']
                current_value = next(stock['current_value'] for stock in turn['stocks'] if stock['name'] == choice)
                
                # 다음 턴의 가치 찾기
                if next_turn_index < len(game_data):
                    next_turn = game_data[next_turn_index]
                    next_value = next(stock['current_value'] for stock in next_turn['stocks'] if stock['name'] == choice)
                else:
                    # 마지막 턴이면 현재 가치의 ±10% 랜덤 변동
                    import random
                    next_value = current_value * (1 + random.uniform(-0.1, 0.1))
                
                # 수익률 계산
                profit_rate = (next_value - current_value) / current_value
                profit = capital * profit_rate
                capital = capital + profit
                
                print(f"\n투자 결과: {choice}에 투자하여 {profit:.1f}원의 {'수익' if profit >= 0 else '손실'}이 발생했습니다.")
                print(f"새로운 자본금: {capital:.1f}원")
            
            # 이벤트 표시
            if turn['event_description'] != "없음":
                print(f"\n[이벤트 발생] {turn['event_description']}")
            
            # 다음 턴으로 넘어가기 전에 대기
            if turn['turn_number'] < len(game_data):
                input("\n엔터 키를 눌러 다음 턴으로 진행하세요...")
        
        # 시뮬레이션 종료
        print("\n===== 시뮬레이션 종료 =====")
        print(f"최종 자본금: {capital:.1f}원")
        initial_capital = 1000
        profit_rate = (capital - initial_capital) / initial_capital * 100
        print(f"최종 수익률: {profit_rate:.1f}%")
        
        if profit_rate > 50:
            print("대단합니다! 투자의 귀재군요! 😃")
        elif profit_rate > 0:
            print("성공적인 투자였습니다! 👍")
        elif profit_rate > -20:
            print("아쉽게도 약간의 손실이 발생했습니다. 다음에 더 좋은 결과가 있을 거예요. 🙂")
        else:
            print("큰 손실이 발생했네요. 다음에는 더 신중하게 투자해보세요. 😢")
        
    except Exception as e:
        print(f"시뮬레이션 중 오류 발생: {e}")

# 시뮬레이션 실행 여부 확인
if 'parsed_game_data' in locals() and parsed_game_data is not None:
    run_simulation_input = input("시뮬레이션을 실행하시겠습니까? (y/n): ")
    if run_simulation_input.lower() == 'y':
        run_simulation(parsed_game_data)